# TP03 — Inverse Kinematics of Robot Manipulators

**GEII3 — Industrie 4.0: Robotique | Lab Session 3**

**Duration**: 2 sessions × 4 hours  
**Prerequisites**: TP02 (Forward Kinematics), linear algebra, trigonometry

---

## Learning Objectives

By the end of this lab (8 hours total), you should be able to:

1. Understand the **inverse kinematics problem** and its challenges

2. Apply **analytical/geometric methods** for simple robots:
   - Geometric approach (triangulation)
   - Algebraic approach (solving equations)
   - Decoupling method (position + orientation)

3. Implement **numerical methods** for complex robots:
   - Jacobian-based (Newton-Raphson)
   - Damped Least Squares (DLS)
   - Pseudo-inverse methods

4. Handle **multiple solutions** and **singularities**

5. Apply IK to **trajectory planning** and **real robot control**

6. Validate solutions with **Robotics Toolbox**

---

## The Inverse Kinematics Problem

### Problem Statement

**Given**: Desired end-effector pose $T_d \in SE(3)$ (position + orientation)

**Find**: Joint configuration $\mathbf{q} = [q_1, q_2, ..., q_n]^T$ such that:

$$T(\mathbf{q}) = T_d$$

where $T(\mathbf{q})$ is the forward kinematics function.

### Key Challenges

1. **Existence**: Does a solution exist? (workspace boundaries)
2. **Uniqueness**: Multiple solutions possible (elbow up/down, etc.)
3. **Complexity**: Generally no closed-form solution for n > 6
4. **Singularities**: Infinite solutions or no solution locally
5. **Joint limits**: Physical constraints on $q_i$

### Solution Approaches

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| **Analytical** | Fast, exact, all solutions | Only for special geometries | 2R, 3R, 6R with spherical wrist |
| **Numerical** | General purpose | Iterative, local minima | Any robot, real-time control |
| **Geometric** | Intuitive, educational | Limited applicability | Simple planar robots |

---

## Lab Organization

### **Session 1** (4 hours)
- **Part I**: Theory & setup (20 min)
- **Exercise 1**: 2R planar - analytical & geometric (50 min)
- **Exercise 2**: 3R planar - algebraic method (60 min)
- **Exercise 3**: Numerical methods introduction (70 min)
- **Exercise 4**: 3R anthropomorphic - decoupling (40 min)

### **Session 2** (4 hours)
- **Exercise 5**: PUMA 560 - analytical with decoupling (80 min)
- **Exercise 6**: Numerical IK for general robots (70 min)
- **Exercise 7**: Trajectory tracking & workspace limits (50 min)
- **Exercise 8**: Comparative analysis & conclusions (40 min)

---

# Part I: Theoretical Background

## 1. Analytical vs Numerical Methods

### Analytical (Closed-Form) Solutions

**Approach**: Solve kinematic equations algebraically

**Requirements**:
- Special geometric structure (e.g., spherical wrist)
- Sufficient equation simplification possible
- Typically requires Pieper's conditions or equivalent

**Example for 2R planar**:

Given target $(x_d, y_d)$:

$$\begin{aligned}
x_d &= L_1\cos(q_1) + L_2\cos(q_1+q_2) \\
y_d &= L_1\sin(q_1) + L_2\sin(q_1+q_2)
\end{aligned}$$

Solution exists if: $|L_1 - L_2| \leq \sqrt{x_d^2 + y_d^2} \leq L_1 + L_2$

### Numerical (Iterative) Solutions

**Approach**: Iteratively minimize error $e = T_d - T(\mathbf{q})$

**Newton-Raphson method**:

$$\mathbf{q}_{k+1} = \mathbf{q}_k + J^{-1}(\mathbf{q}_k) \cdot \mathbf{e}_k$$

where $J$ is the Jacobian matrix.

**Damped Least Squares (Levenberg-Marquardt)**:

$$\Delta\mathbf{q} = (J^T J + \lambda I)^{-1} J^T \mathbf{e}$$

where $\lambda$ is damping factor (helps near singularities).

---

## 2. Geometric Decoupling

For 6-DOF robots with **spherical wrist** (last 3 axes intersect):

1. **Position subproblem**: Use first 3 joints to reach wrist center
2. **Orientation subproblem**: Use last 3 joints to achieve orientation

This reduces a 6-DOF problem to two 3-DOF problems!

---

## Setup and Installation

In [ ]:
# Uncomment to install:
# !pip install -q roboticstoolbox-python spatialmath-python matplotlib scipy

import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, atan2, sqrt, acos, asin
from scipy.optimize import fsolve, minimize
from scipy.linalg import pinv, norm

try:
    import roboticstoolbox as rtb
    from spatialmath import SE3, SO3
    RTB_AVAILABLE = True
    print(f"✓ RTB version: {rtb.__version__}")
except ImportError:
    RTB_AVAILABLE = False
    print("⚠ RTB not available")

np.set_printoptions(precision=4, suppress=True)
print("✓ Libraries loaded\n")

## Helper Functions

Import FK functions from TP02 and add IK-specific utilities.

In [ ]:
# ============================================================================
# FORWARD KINEMATICS (from TP02)
# ============================================================================

def dh_standard(theta, d, a, alpha):
    """Standard DH transformation."""
    ct, st = cos(theta), sin(theta)
    ca, sa = cos(alpha), sin(alpha)
    return np.array([
        [ct, -st*ca,  st*sa, a*ct],
        [st,  ct*ca, -ct*sa, a*st],
        [0,      sa,     ca,    d],
        [0,       0,      0,    1]
    ])

def fk_dh(dh_params, q, convention='standard'):
    """Forward kinematics using DH parameters."""
    T = np.eye(4)
    T_list = [T.copy()]
    
    for i, (theta_off, d_off, a, alpha, jtype) in enumerate(dh_params):
        theta = (theta_off + q[i]) if jtype == 'r' else theta_off
        d = d_off if jtype == 'r' else (d_off + q[i])
        
        T_i = dh_standard(theta, d, a, alpha)
        T = T @ T_i
        T_list.append(T.copy())
    
    return T, T_list

def extract_position(T):
    """Extract position from transformation matrix."""
    return T[:3, 3]

def extract_rotation(T):
    """Extract rotation from transformation matrix."""
    return T[:3, :3]


# ============================================================================
# INVERSE KINEMATICS UTILITIES
# ============================================================================

def pose_error(T_current, T_desired):
    """
    Compute pose error between current and desired transformations.
    
    Returns 6D error vector: [position_error (3), orientation_error (3)]
    """
    # Position error
    p_error = extract_position(T_desired) - extract_position(T_current)
    
    # Orientation error (simplified - axis-angle representation)
    R_current = extract_rotation(T_current)
    R_desired = extract_rotation(T_desired)
    R_error = R_desired @ R_current.T
    
    # Convert to axis-angle (simplified)
    theta = acos((np.trace(R_error) - 1) / 2)
    if abs(theta) < 1e-6:
        o_error = np.zeros(3)
    else:
        axis = np.array([
            R_error[2,1] - R_error[1,2],
            R_error[0,2] - R_error[2,0],
            R_error[1,0] - R_error[0,1]
        ]) / (2 * sin(theta))
        o_error = theta * axis
    
    return np.concatenate([p_error, o_error])


def numerical_jacobian(dh_params, q, epsilon=1e-6):
    """
    Compute numerical Jacobian (6×n) using finite differences.
    Returns J such that dx = J * dq
    """
    n = len(q)
    J = np.zeros((6, n))
    
    T_0, _ = fk_dh(dh_params, q)
    p_0 = extract_position(T_0)
    R_0 = extract_rotation(T_0)
    
    for i in range(n):
        q_delta = q.copy()
        q_delta[i] += epsilon
        
        T_delta, _ = fk_dh(dh_params, q_delta)
        p_delta = extract_position(T_delta)
        R_delta = extract_rotation(T_delta)
        
        # Linear velocity
        J[:3, i] = (p_delta - p_0) / epsilon
        
        # Angular velocity (simplified)
        dR = (R_delta - R_0) / epsilon
        J[3, i] = (dR[2,1] - dR[1,2]) / 2
        J[4, i] = (dR[0,2] - dR[2,0]) / 2
        J[5, i] = (dR[1,0] - dR[0,1]) / 2
    
    return J


def normalize_angle(angle):
    """Normalize angle to [-pi, pi]."""
    return np.arctan2(np.sin(angle), np.cos(angle))


def check_joint_limits(q, limits):
    """
    Check if joint configuration respects limits.
    
    Parameters:
    - q: joint configuration
    - limits: list of (min, max) tuples
    
    Returns: True if within limits
    """
    for i, (qmin, qmax) in enumerate(limits):
        if q[i] < qmin or q[i] > qmax:
            return False
    return True


# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_robot_2d(T_list, ax=None, title="Robot Configuration", target=None):
    """Visualize robot in 2D with optional target."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    x = [T[0, 3] for T in T_list]
    y = [T[1, 3] for T in T_list]
    
    ax.plot(x, y, 'o-', linewidth=2, markersize=8, label='Robot', color='blue')
    ax.plot(x[0], y[0], 'gs', markersize=12, label='Base')
    ax.plot(x[-1], y[-1], 'r*', markersize=15, label='End-effector')
    
    # Target position
    if target is not None:
        ax.plot(target[0], target[1], 'mx', markersize=15, 
                markeredgewidth=3, label='Target')
        # Draw line from EE to target
        ax.plot([x[-1], target[0]], [y[-1], target[1]], 
                'r--', alpha=0.5, linewidth=1)
    
    # End-effector frame
    T_end = T_list[-1]
    scale = 0.1
    ax.arrow(T_end[0,3], T_end[1,3], scale*T_end[0,0], scale*T_end[1,0],
             head_width=0.03, head_length=0.02, fc='red', ec='red', alpha=0.7)
    ax.arrow(T_end[0,3], T_end[1,3], scale*T_end[0,1], scale*T_end[1,1],
             head_width=0.03, head_length=0.02, fc='green', ec='green', alpha=0.7)
    
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    
    return ax


def compare_solutions(dh_params, solutions, target_pos, title="IK Solutions"):
    """
    Visualize multiple IK solutions for the same target.
    """
    n_sols = len(solutions)
    fig, axes = plt.subplots(1, n_sols, figsize=(6*n_sols, 5))
    
    if n_sols == 1:
        axes = [axes]
    
    for idx, (q, label) in enumerate(solutions):
        _, T_list = fk_dh(dh_params, q)
        plot_robot_2d(T_list, axes[idx], 
                      title=f"{label}\nq={np.rad2deg(q).astype(int)}°",
                      target=target_pos)
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("✓ Helper functions defined\n")

---

# Exercise 1: 2R Planar Robot - Analytical IK

**Estimated time**: 50 minutes (Session 1)

## Problem Description

2-DOF planar robot with:
- $L_1 = 0.4$ m
- $L_2 = 0.3$ m

**Task**: Given target position $(x_d, y_d)$, find joint angles $(q_1, q_2)$.

## Task 1.1: Geometric Solution (20 min)

**Method**: Use law of cosines and geometry.

### Your derivation:

**Step 1**: Compute distance to target:
$$D = \sqrt{x_d^2 + y_d^2}$$

**Step 2**: Check reachability:
$$|L_1 - L_2| \leq D \leq L_1 + L_2$$

**Step 3**: Find $q_2$ using law of cosines:
$$\cos(q_2) = \frac{x_d^2 + y_d^2 - L_1^2 - L_2^2}{2 L_1 L_2}$$

**Two solutions**: $q_2 = \pm \arccos(\cos(q_2))$ (elbow up/down)

**Step 4**: Find $q_1$:
$$q_1 = \text{atan2}(y_d, x_d) - \text{atan2}(L_2\sin(q_2), L_1 + L_2\cos(q_2))$$

---

## Task 1.2: Implementation (30 min)

In [ ]:
# Robot parameters
L1 = 0.4
L2 = 0.3

# DH parameters
dh_2r = [
    (0, 0, L1, 0, 'r'),
    (0, 0, L2, 0, 'r'),
]

def ik_2r_analytical(x_target, y_target, L1, L2, elbow='up'):
    """
    Analytical IK for 2R planar robot.
    
    Parameters:
    - x_target, y_target: desired position
    - L1, L2: link lengths
    - elbow: 'up' or 'down'
    
    Returns:
    - q: [q1, q2] if solution exists, None otherwise
    """
    # TODO: Implement based on your derivation
    
    # Step 1: Distance to target
    D = sqrt(x_target**2 + y_target**2)
    
    # Step 2: Check reachability
    if D > (L1 + L2) or D < abs(L1 - L2):
        print(f"Target unreachable! D={D:.3f}, range=[{abs(L1-L2):.3f}, {L1+L2:.3f}]")
        return None
    
    # Step 3: Solve for q2
    cos_q2 = (x_target**2 + y_target**2 - L1**2 - L2**2) / (2 * L1 * L2)
    
    # Numerical safety
    cos_q2 = np.clip(cos_q2, -1, 1)
    
    if elbow == 'up':
        q2 = acos(cos_q2)  # Positive angle
    else:
        q2 = -acos(cos_q2)  # Negative angle (elbow down)
    
    # Step 4: Solve for q1
    alpha = atan2(y_target, x_target)
    beta = atan2(L2 * sin(q2), L1 + L2 * cos(q2))
    q1 = alpha - beta
    
    return np.array([q1, q2])


# Test cases
print("\n" + "="*70)
print("  2R PLANAR ROBOT - ANALYTICAL IK")
print("="*70)

test_targets = [
    (0.5, 0.3, "Reachable point"),
    (0.6, 0.2, "Near workspace boundary"),
    (0.0, 0.7, "Straight up"),
]

solutions = []

for x_t, y_t, desc in test_targets:
    print(f"\n[Target: ({x_t}, {y_t}) - {desc}]")
    
    # Both elbow configurations
    q_up = ik_2r_analytical(x_t, y_t, L1, L2, elbow='up')
    q_down = ik_2r_analytical(x_t, y_t, L1, L2, elbow='down')
    
    if q_up is not None:
        print(f"  Elbow UP:   q = {np.rad2deg(q_up)}°")
        
        # Verify with FK
        T_up, _ = fk_dh(dh_2r, q_up)
        pos_up = extract_position(T_up)
        error_up = np.linalg.norm([pos_up[0] - x_t, pos_up[1] - y_t])
        print(f"              Reached: ({pos_up[0]:.4f}, {pos_up[1]:.4f})")
        print(f"              Error: {error_up:.2e} m")
        
        solutions.append((q_up, f"Elbow UP\nTarget: ({x_t}, {y_t})"))
    
    if q_down is not None:
        print(f"  Elbow DOWN: q = {np.rad2deg(q_down)}°")
        
        T_down, _ = fk_dh(dh_2r, q_down)
        pos_down = extract_position(T_down)
        error_down = np.linalg.norm([pos_down[0] - x_t, pos_down[1] - y_t])
        print(f"              Reached: ({pos_down[0]:.4f}, {pos_down[1]:.4f})")
        print(f"              Error: {error_down:.2e} m")
        
        solutions.append((q_down, f"Elbow DOWN\nTarget: ({x_t}, {y_t})"))

# Visualize first target's solutions
if len(solutions) >= 2:
    compare_solutions(dh_2r, solutions[:2], (test_targets[0][0], test_targets[0][1]),
                     title="2R Planar - Elbow Up vs Down")

## Task 1.3: Validation with RTB (10 min)

In [ ]:
if RTB_AVAILABLE:
    from roboticstoolbox import DHRobot, RevoluteDH
    
    robot_2r = DHRobot([
        RevoluteDH(a=0,  d=0, alpha=0),
        RevoluteDH(a=L1, d=0, alpha=0)
    ], name="2R Planar")
    
    print("\n" + "="*70)
    print("  VALIDATION WITH RTB")
    print("="*70)
    
    # Test one target
    x_t, y_t = 0.5, 0.3
    T_target = SE3(x_t, y_t, 0)
    
    # RTB IK (numerical)
    q_rtb = robot_2r.ikine_LM(T_target, mask=[1,1,0,0,0,0])[0]  # Only position
    
    # Our analytical
    q_ours = ik_2r_analytical(x_t, y_t, L1, L2, elbow='up')
    
    print(f"\nTarget: ({x_t}, {y_t})")
    print(f"Our solution:  {np.rad2deg(q_ours)}°")
    print(f"RTB solution:  {np.rad2deg(q_rtb)}°")
    print(f"Difference:    {np.rad2deg(q_rtb - q_ours)}°")
    
else:
    print("RTB not available")